Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

# Conv

In [ ]:
import findspark
import os

### Versions

In [ ]:
cuda_version = 'cuda11'
hadoop_version = '3'
java_version = '8'
rapids_version = '23.10.0-SNAPSHOT'
scala_version = '2.12'
spark_version = '3.3.2'

### Environment Variables

In [ ]:
os.environ['JAVA_HOME'] = f"/usr/lib/jvm/java-{java_version}-openjdk-amd64"
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['TZ'] = 'UTC'

In [ ]:
home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
m2_local_repo = f"{home_dir}/.m2/repository"
groupId = "com.nvidia"
artifactId = f"rapids-4-spark_{scala_version}"
dist_jar = f"{artifactId}-{rapids_version}-{cuda_version}.jar"
spark_home = f"{home_dir}/dist/spark-{spark_version}-bin-hadoop{hadoop_version}"
# spark_home = f"{home_dir}/gits/apache/spark"

### Find Spark 

In [ ]:
findspark.init(spark_home = spark_home)
findspark.add_jars(f"{m2_local_repo}/com/nvidia/{artifactId}/{rapids_version}/{dist_jar}")

import pyspark

### Configure Spark

In [ ]:
cores_per_exec = 1
jdwp = '-agentlib:jdwp=transport=dt_socket,server=y,suspend=n,address=5005'
spark_master = f"local[{cores_per_exec}]"
spark_builder = pyspark.sql.SparkSession.builder
spark_builder.config('spark.app.name', 'RAPIDS PySpark Notebook')
spark_builder.config('spark.driver.extraJavaOptions', f"-Dai.rapids.cudf.preserve-dependencies=true {jdwp}")
spark_builder.config('spark.master', spark_master)
spark_builder.config('spark.plugins', 'com.nvidia.spark.SQLPlugin')
spark_builder.config('spark.rapids.sql.explain', 'ALL')

spark = spark_builder.getOrCreate()

# Test Scenario

In [ ]:
df = spark.createDataFrame([
    ('gera', 2), 
    ('231', 10), 
    ('0xDEADBEEF', 16),
    ('915270249210239718', 10),
    ], 'a string, b integer')
df.printSchema()
df.createOrReplaceTempView("TBL")
df.show(truncate = False)

In [ ]:
from pyspark.sql import functions as fun
df.select('a', fun.conv('a', 10, 16)).collect()

In [ ]:
from enum import Enum
class Chip(Enum):
    CPU = 1
    GPU = 2

In [ ]:
def test_case(chip):
    print(f"##### TEST {chip=}\n")
    spark.conf.set('spark.rapids.sql.enabled', chip == Chip.GPU)
    print("### original")
    print(df.collect())
    print("### non-literal from_base")
    print(spark.sql("SELECT CONV(a, b, 16) FROM TBL").collect())
    
    print("### literal but one not 10 or 16")
    print(df.select(pyspark.sql.functions.conv('a', 10, 2)).collect())
    print(df.select(pyspark.sql.functions.conv('a', 16, 5)).collect())

    print("### 10 -> 16")
    print(df.select(pyspark.sql.functions.conv('a', 10, 16)).collect())
    print("### 16 -> 10")
    print(df.select(pyspark.sql.functions.conv('a', 16, 10)).collect())

### GPU

In [ ]:
test_case(chip = Chip.GPU)

### CPU